# Transformer_Korean_Chatbot

In [ ]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

chat_data = pd.read_csv('ChatBotData.csv')
chat_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
chat_data.shape

(11823, 3)

## 데이터 전처리

In [ ]:
#불필요한 문장부호 삭제
import string
exclude=set(string.punctuation)
chat_data.Q=chat_data.Q.apply(lambda x:''.join (ch for ch in x if ch not in exclude))
chat_data.A=chat_data.A.apply(lambda x:''.join (ch for ch in x if ch not in exclude))
print(chat_data.head())

                 Q           A  label
0            12시 땡   하루가 또 가네요      0
1      1지망 학교 떨어졌어    위로해 드립니다      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠      0
4          PPL 심하네   눈살이 찌푸려지죠      0


In [ ]:
#!pip install tensorflow_datasets
import tensorflow_datasets as tfds
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    chat_data.Q+chat_data.A, target_vocab_size=2**13)
print(tokenizer.vocab_size)

8510


총 어휘 8510개

In [ ]:
ex_1=tokenizer.encode(chat_data.loc[20,'Q'])
print(ex_1)
ex_2=tokenizer.decode(ex_1)
print(ex_2)

[5871, 590, 3523, 127, 695, 3756, 833]
가스비 비싼데 감기 걸리겠어


In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

VOCAB_SIZE = tokenizer.vocab_size + 2 #2개의 token이 추가됨
print(VOCAB_SIZE)

8512


In [ ]:
tokenized_inputs, tokenized_outputs = [], []

for (sentence1, sentence2) in zip(chat_data.Q, chat_data.A):
    # decoder(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = tokenizer.encode(sentence1)
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2) 

print(tokenized_inputs[:5])
print(tokenized_outputs[:5])

[[8251, 1153, 3075], [8303, 47, 906, 8286, 976, 1715], [8305, 1433, 4683, 8286, 3666, 87], [8305, 1433, 4683, 8286, 1284, 3666, 87], [8334, 8334, 8330, 8286, 4213]]
[[8510, 3857, 64, 8230, 8511], [8510, 1824, 5586, 8511], [8510, 3409, 762, 120, 8511], [8510, 3409, 762, 120, 8511], [8510, 964, 2300, 1490, 2180, 4436, 40, 8511]]


In [ ]:
# 최대길이 20, 29
import numpy as np
lenth_list=[]
for l in tokenized_inputs:
    lenth_list.append(len(l))
q_lenth_max=np.max(lenth_list)
print(q_lenth_max)

lenth_list=[]
for l in tokenized_outputs:
    lenth_list.append(len(l))
a_lenth_max=np.max(lenth_list)
print(a_lenth_max)


20
29


In [ ]:
# 패딩
import tensorflow as tf

max_length_q=20
questions = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=max_length_q, padding='post')

max_length_a=29
answers = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=max_length_a, padding='post')

In [ ]:
print(questions[:2])
print(answers[:2])

[[8251 1153 3075    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [8303   47  906 8286  976 1715    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
[[8510 3857   64 8230 8511    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [8510 1824 5586 8511    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]]


In [ ]:
print(tokenizer.decode(questions[0]))
print(questions.shape)
print(answers.shape)

12시 땡
(11823, 20)
(11823, 29)


In [ ]:
import tensorflow as tf
data_set=tf.data.Dataset.from_tensor_slices(((questions,answers[:,:-1]),answers[:,1:]))
data_final=data_set.shuffle(500).batch(32).prefetch(32)
ds_test=next(iter(data_final))
print(np.array(ds_test[0][0].shape))
print(np.array(ds_test[1]).shape)
print(np.array(ds_test[0][1].shape))

[32 20]
(32, 28)
[32 28]


## 트랜스포머 모형

In [ ]:
import tensorflow as tf
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)#(1,position,d_model)

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  #x:tokenized words with padding for each sentence 1D텐서 [...,0,0,0]
  # add extra dimensions to add the padding
  # to the attention logits.
  return mask[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
def create_look_ahead_mask(x):
    seq_len=tf.shape(x)[1]
    look_ahead_mask=1-tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0)
    padding_mask=create_padding_mask(x)
    return tf.maximum(look_ahead_mask,padding_mask)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth) (number of heads,ts,64)
    k: key shape == (..., seq_len_k, depth)   (number of heads,ts,64)
    v: value shape == (..., seq_len_v, depth_v) (number of heads,ts,64)
    mask: Float tensor with shape broadcastable  (1,1,ts) or (ts,ts)
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k) (number of heads,ts,ts)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v) (number of heads,ts,64)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads  #8
    self.d_model = d_model #512

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model) #(ts,512)
    self.wk = tf.keras.layers.Dense(d_model) #(ts,512)
    self.wv = tf.keras.layers.Dense(d_model) #(ts,512)

    self.dense = tf.keras.layers.Dense(d_model) # scale_dot_attention후 concatenate한 attention (ts,512)를 Dense에 통과하기 위함

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    (batch,ts,512)를 (batch,ts,heads(8), depth(64))로 reshape한후 (batch,heads,ts,depth)로 반환하는 함수
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, ts, 512) 여기에서 512=d_model=embedding_size
    k = self.wk(k)  # (batch_size, ts, 512)
    v = self.wv(v)  # (batch_size, ts, 512)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth) depth=64, num_heads=8,seq_len_q=ts 
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth) depth=64, num_heads=8,seq_len_k=ts 
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth) depth=64, num_heads=8,seq_len_v=ts 

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff) seq_len=ts
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model) d_model=512
  ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model) input_seq_len=input_ts, d_model=512
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model) x: decoder input

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model) 
    attn1 = self.dropout1(attn1)                   #targer_seq_len=decoder_input_ts
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask) 
    attn2 = self.dropout2(attn2)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model) #target_seq_len=decoder_input_ts

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    seq_len = tf.shape(x)[1]  #x는 tokenized sentence with padding

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output,look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [ ]:
def transformer(num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, ts_input, ts_target, rate=0.1):
    
    enc_input = tf.keras.Input(shape=(None,))  
    dec_input = tf.keras.Input(shape=(None,))
    
    enc_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    look_ahead_mask=tf.keras.layers.Lambda(create_look_ahead_mask,output_shape=(1,None,None))(dec_input)
    dec_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    
    enc_output=Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, ts_input, rate)(enc_input,enc_padding_mask)
    dec_output,_ =Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, ts_target, rate)(dec_input,enc_output,look_ahead_mask,dec_padding_mask)
    outputs=tf.keras.layers.Dense(target_vocab_size)(dec_output)
    
    return tf.keras.Model(inputs=[enc_input, dec_input], outputs=outputs)
    

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
def loss_function(y_true, y_pred):
  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  loss_ = loss_object(y_true, y_pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask) #padding을 제외한 loss


def accuracy_function(y_true, y_pred):
  y_hat=tf.cast(tf.argmax(y_pred,axis=2), dtype=tf.float32) #y_true가 float32이기 때문
  accuracies = tf.equal(y_true, y_hat)
  #accuracies = tf.equal(y_true, tf.argmax(y_pred, axis=2)) #True, False로 출력

  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  accuracies = tf.math.logical_and(mask, accuracies) #둘다 true일 때만 True

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

def accuracy(y_true,y_pred):
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


## 모델1:

**d_model** :트랜스포머의 인코더와 디코더에서의 정해진 입력과 출력의 크기 , 임베딩 벡터의 차원

**num_layer** : 인코더와 디코더가 총 몇 층으로 구성되었는지를 의미

**num_head** : 텐션을 사용할 때, 한 번 하는 것 보다 여러 개로 분할해서 병렬로 어텐션을 수행하고 결과값을 다시 하나로 합치는 방식을 택했습니다. 이때 이 병렬의 개수

**dff** : 트랜스포머 내부에는 피드 포워드 신경망이 존재하며 해당 신경망의 은닉층의 크기

In [ ]:
model=transformer(num_layers=3,d_model=128,num_heads=4,dff=256, 
                  input_vocab_size=VOCAB_SIZE-2, target_vocab_size=VOCAB_SIZE,ts_input=20,ts_target=28,rate=0.1)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])
model.fit(data_final,epochs=30)

Epoch 1/30
370/370 [==============================] - 30s 46ms/step - loss: 6.8055 - accuracy_function: 0.1771
Epoch 2/30
370/370 [==============================] - 17s 45ms/step - loss: 5.9313 - accuracy_function: 0.1903
Epoch 3/30
370/370 [==============================] - 17s 46ms/step - loss: 5.6811 - accuracy_function: 0.2004
Epoch 4/30
370/370 [==============================] - 18s 48ms/step - loss: 5.5260 - accuracy_function: 0.2082
Epoch 5/30
370/370 [==============================] - 17s 45ms/step - loss: 5.3869 - accuracy_function: 0.2168
Epoch 6/30
370/370 [==============================] - 25s 67ms/step - loss: 5.2139 - accuracy_function: 0.2262
Epoch 7/30
370/370 [==============================] - 17s 45ms/step - loss: 5.0480 - accuracy_function: 0.2344
Epoch 8/30
370/370 [==============================] - 17s 45ms/step - loss: 4.9147 - accuracy_function: 0.2415
Epoch 9/30
370/370 [==============================] - 17s 46ms/step - loss: 4.8070 - accuracy_function: 0.2459
E

## 모델2: num_layers = 1

In [ ]:
model=transformer(num_layers=1,d_model=128,num_heads=4,dff=256, 
                  input_vocab_size=VOCAB_SIZE-2, target_vocab_size=VOCAB_SIZE,ts_input=20,ts_target=28,rate=0.1)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])
model.fit(data_final,epochs=30)

Epoch 1/30
370/370 [==============================] - 16s 23ms/step - loss: 6.5594 - accuracy_function: 0.1953
Epoch 2/30
370/370 [==============================] - 8s 23ms/step - loss: 5.1338 - accuracy_function: 0.2658
Epoch 3/30
370/370 [==============================] - 8s 23ms/step - loss: 4.0328 - accuracy_function: 0.3655
Epoch 4/30
370/370 [==============================] - 8s 23ms/step - loss: 3.0899 - accuracy_function: 0.4579
Epoch 5/30
370/370 [==============================] - 9s 23ms/step - loss: 2.3425 - accuracy_function: 0.5491
Epoch 6/30
370/370 [==============================] - 9s 23ms/step - loss: 1.7275 - accuracy_function: 0.6446
Epoch 7/30
370/370 [==============================] - 9s 23ms/step - loss: 1.2672 - accuracy_function: 0.7220
Epoch 8/30
370/370 [==============================] - 9s 24ms/step - loss: 0.9423 - accuracy_function: 0.7808
Epoch 9/30
370/370 [==============================] - 9s 24ms/step - loss: 0.7069 - accuracy_function: 0.8273
Epoch 10/

## 모델3: dff = 128 (은닉층 크기)

In [ ]:
model=transformer(num_layers=1,d_model=128,num_heads=4,dff=128, 
                  input_vocab_size=VOCAB_SIZE-2, target_vocab_size=VOCAB_SIZE,ts_input=20,ts_target=28,rate=0.1)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])
model.fit(data_final,epochs=30)

Epoch 1/30
370/370 [==============================] - 18s 33ms/step - loss: 6.5414 - accuracy_function: 0.1959
Epoch 2/30
370/370 [==============================] - 8s 22ms/step - loss: 5.0832 - accuracy_function: 0.2703
Epoch 3/30
370/370 [==============================] - 8s 22ms/step - loss: 3.9720 - accuracy_function: 0.3687
Epoch 4/30
370/370 [==============================] - 8s 22ms/step - loss: 3.0387 - accuracy_function: 0.4591
Epoch 5/30
370/370 [==============================] - 8s 22ms/step - loss: 2.2919 - accuracy_function: 0.5556
Epoch 6/30
370/370 [==============================] - 8s 23ms/step - loss: 1.6868 - accuracy_function: 0.6478
Epoch 7/30
370/370 [==============================] - 9s 23ms/step - loss: 1.2220 - accuracy_function: 0.7266
Epoch 8/30
370/370 [==============================] - 8s 22ms/step - loss: 0.8880 - accuracy_function: 0.7895
Epoch 9/30
370/370 [==============================] - 9s 24ms/step - loss: 0.6683 - accuracy_function: 0.8349
Epoch 10/

## 결과

In [ ]:
def evaluate(sentence):
  sentence_1 = preprocess_sentence(sentence)
  token_sentence=[]
  t_sentence=tokenizer.encode(sentence_1)
  token_sentence.append(t_sentence)
  encoder_input=tf.keras.preprocessing.sequence.pad_sequences(token_sentence, maxlen=20, padding='post')
  print(encoder_input)
  decoder_input_pred=np.zeros((1,max_length_a-1),dtype='int32')
  word=START_TOKEN[0]
  for i in range(max_length_a-1):
     decoder_input_pred[:,i]=word
     pred=model.predict((encoder_input,decoder_input_pred))
     t=np.argmax(pred[0][i])
     word=t
     if word == END_TOKEN:
          break
  print(pred.shape)
  print(decoder_input_pred)
  
  return encoder_input, decoder_input_pred
  

In [ ]:
def predict(sentence):
  encode_input, decode_input = evaluate(sentence)
  
  t=model.predict([encode_input,decode_input])
  prediction=np.argmax(t,axis=2)
  print(prediction)
  print(prediction.shape)
  prediction=tf.squeeze(prediction,axis=0)  
  #prediction_sentence=tokenizer.decode(prediction)
  
  predicted_sentence=[]
  for i in prediction:
    if i>8510:
        break
    else:
        predicted_sentence.append(tokenizer.decode([i]))
    
        
  #predicted_sentence = tokenizer.decode(
    #[i for i in list(prediction) if i < 8511])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
exclude=set(string.punctuation)
def preprocess_sentence(x):
    t=[]
    for i in range(len(x)):
        if x[i] not in exclude:
            t.append(x[i])
    #print(t)
    x3=''
    for i in range(len(t)):
        x3=x3+t[i]+''
    return x3

sentence='피곤하고 재미없다.'
sentence_1=preprocess_sentence(sentence)
print(sentence_1)

predict(sentence)

피곤하고 재미없다
[[4783   38 6460   70    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 28, 8512)
[[8510 3812  108 4860  188 3610 7047  209 1551   59   28    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
[[3812  108 4860  188 3610 7047  209 1551   59   28 8511   59   59   59
   709  709  709  709  709  709   59   59  709  709  709  709  709  709]]
(1, 28)
Input: 피곤하고 재미없다.
Output: ['활', '기', '찬 ', '사람을 ', '만나보', '시면 ', '생각이 ', '바뀔 ', '수도 ', '있어요']


['활', '기', '찬 ', '사람을 ', '만나보', '시면 ', '생각이 ', '바뀔 ', '수도 ', '있어요']

In [ ]:
predict('3박4일 놀러가고 싶다') 
predict('카페 갈래')

[[8305 1433 4683 8286 3666   87    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 28, 8512)
[[8510 3409  762  120    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
[[3409  762  120 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511
  8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511]]
(1, 28)
Input: 3박4일 놀러가고 싶다
Output: ['여행은 ', '언제나 ', '좋죠']
[[1770  736  396    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 28, 8512)
[[8510 1770  314  120    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
[[1770  314  120 8511 8286 8286 8286 8286 8286 8286 8286 8286 8511 8511
  8286 8286 8286 8286 8286 8511 8511 8511 8286 8286 8286 8286 8286 8286]]
(1, 28)
Input: 카페 갈래
Output: ['카페 ', '데이트 ', '좋죠']


['카페 ', '데이트 ', '좋죠']

In [ ]:
predict('12시 땡')
predict('PPL 심하네')

[[8251 1153 3075    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 28, 8512)
[[8510 3857   64 8230    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
[[3857   64 8230 8511   22   22   22 8511 8511 8511 8511 8511 8511 8511
    31   31 8511 8511 8511 8511   31   31   31 8511   22   96   96   96]]
(1, 28)
Input: 12시 땡
Output: ['하루가 ', '또 ', '가네요']
[[8334 8334 8330 8286 4213    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 28, 8512)
[[8510  964 2300 1490 2180 4436   40    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
[[ 964 2300 1490 2180 4436   40 8511 8286 8286 8286 2180 2180 1524 1524
  1490 2180 2180 2180 2180 8286 8286    2    2   22   22 2688 2688 8286]]
(1, 28)
Input: PPL 심하네
Output: ['눈', '살이 ', '찌', '푸', '려지', '죠']


['눈', '살이 ', '찌', '푸', '려지', '죠']

In [ ]:
predict('밥 먹어라')

[[ 486 1437  280    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 28, 8512)
[[8510 6766 4652    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]
[[6766 4652 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511
  8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511 8511]]
(1, 28)
Input: 밥 먹어라
Output: ['연락두절돼서 ', '걱정되겠네요']


['연락두절돼서 ', '걱정되겠네요']